# Facial Recognition

In [1]:
import os
import face_recognition
import cv2
import pyttsx3

In [2]:
engine = pyttsx3.init()
rate = engine.getProperty('rate')   # getting details of current speaking rate
engine.setProperty('rate', 125)

In [3]:
def stored_encodings():
    dir = "C:/Users/User/Documents/capstone/website_practice/upload/train/"
    known_face_names = os.listdir(dir)
    known_face_encodings = []
    for name in known_face_names:
        name = face_recognition.load_image_file(dir + name)
        encodings = face_recognition.face_encodings(name)[0]
        known_face_encodings.append(encodings)
    return known_face_encodings, known_face_names

In [4]:
def capture_img():
    """capturing an image with the camera"""
    cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
    check, frame = cap.read()
    cap.release()
    if check:
        return frame
    else:
        if engine._inLoop:
            engine.endLoop()
        engine.say("Cannot record")
        engine.runAndWait()
        return 0

In [5]:
def recog():
    if engine._inLoop:
        engine.endLoop()
    photo = capture_img()
    name_of_person = 'Unknown face'
    #if photo.all() != 0:
    all_face_locations = []
    all_face_encodings = []
    all_matches = []
    known_face_encodings, known_face_names = stored_encodings()
    current_frame_small = cv2.resize(photo,(0,0),fx=0.25,fy=0.25)
    all_face_locations = face_recognition.face_locations(current_frame_small)
    all_face_encodings = face_recognition.face_encodings(current_frame_small,all_face_locations)
    for current_face_location,current_face_encoding in zip(all_face_locations,all_face_encodings):
        top_pos,right_pos,bottom_pos,left_pos = current_face_location
        top_pos = top_pos*4
        right_pos = right_pos*4
        bottom_pos = bottom_pos*4
        left_pos = left_pos*4
        all_matches = face_recognition.compare_faces(known_face_encodings, current_face_encoding)
        
    if True in all_matches:
        first_match_index = all_matches.index(True)
        name = known_face_names[first_match_index]
        name_of_person = name.split(".", 1)        
        cv2.rectangle(photo,(left_pos,top_pos),(right_pos,bottom_pos),(255,0,0),2)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(photo, name_of_person[0], (left_pos,bottom_pos), font, 1, (255,255,255),2)     
        cv2.imwrite("C:/Users/User/Documents/capstone/website_practice/static/capture.png", photo) 
        engine.say(f"There is {name_of_person[0]} in front of you")
        engine.runAndWait()
        return name_of_person[0]
    else: 
        photo = cv2.imread("C:/Users/User/Documents/capstone/website_practice/static/Unknown.jpg")
        cv2.imwrite("C:/Users/User/Documents/capstone/website_practice/static/capture.png", photo)
        engine.say("Cannot Recognize")
        engine.runAndWait()
        return name_of_person
    return redirect(url_for('detect_face'))

# Object Detection

In [6]:
import os
import cv2
from google.cloud import vision
from gtts import gTTS
import numpy
from playsound import playsound
import argparse
from PIL import Image, ImageDraw


In [7]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "object_detection.json"


def speaker(text):
    """returning results with the speaker"""
    try:
        gTTS(text=text, lang="en").save(f"{text.split()[0]}.mp3")
        playsound(f"{text.split()[0]}.mp3")
        os.remove(f"{text.split()[0]}.mp3")
    except Exception:
        print("Could not activate the speaker")


def capture_img():
    """capturing an image with the camera"""
    cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)  # make sure back camera is accessed, not front
    check, frame = cap.read()
    cap.release()
    if check:
        return frame
    else:
        speaker("I didn't manage to take a picture")


def save_img(frame):
    """saving the image in the file 'capture.png'"""
    frame.dtype = numpy.uint8
    cv2.imwrite("capture.png", frame)


def google_vision(photo):
    """detecting objects on the photo with Google Vision API"""
    google_vision.vects = []
    objects = []
    result = ''
    try:
        client = vision.ImageAnnotatorClient()
        with open(photo, 'rb') as image_file:
            content = image_file.read()
        image = vision.Image(content=content)
        objects = client.object_localization(image=image).localized_object_annotations
        if objects:
            for num, object_ in enumerate(objects):
                if len(objects) > 1 and num == len(objects) - 2:
                    result += object_.name + ' and '
                else:
                    result += object_.name+', '
            speaker(f"There is probably {result} in front of you")
        else:
            speaker("I could not find anything.")
    except Exception:
        speaker("I could not perform an image search.")
        
    for ob in objects:
        for vertex in ob.bounding_poly.normalized_vertices:

            google_vision.vects.append([vertex.x, vertex.y])
    im = Image.open(photo)
    width , height = im.size
    draw = ImageDraw.Draw(im)
    for vertex in objects:
        draw.polygon([
            vertex.bounding_poly.normalized_vertices[0].x *width, vertex.bounding_poly.normalized_vertices[0].y*height,
            vertex.bounding_poly.normalized_vertices[1].x *width, vertex.bounding_poly.normalized_vertices[1].y*height,
            vertex.bounding_poly.normalized_vertices[2].x *width, vertex.bounding_poly.normalized_vertices[2].y*height,
            vertex.bounding_poly.normalized_vertices[3].x *width, vertex.bounding_poly.normalized_vertices[3].y*height], None, 'red')
    im.save('static/output.jpg', 'JPEG')
    return result
    
    
def main():
    """
    This Object Detector gets user's voice input (code word "object")
    and takes a photo with the device's default camera,
    then detects objects on it with Google Vision API
    and returns audio output stating which object is in front of the user.
    """
    photo = capture_img()
    if photo is not None:
        save_img(photo)
        return google_vision("capture.png")

# Integration in flask

## Flask Configuration

In [8]:
import numpy as np
from flask import Flask, render_template,request, redirect, url_for
import sys
app = Flask(__name__)

In [9]:

UPLOAD_FOLDER = 'C:/Users/User/Documents/capstone/website_practice/upload/train'
ALLOWED_EXTENSIONS = set(['png', 'jpg', 'jpeg'])
app.config['SEND_FILE_MAX_AGE_DEFAULT'] = 0
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

In [10]:
@app.after_request
def add_header(r):
    r.headers["Cache-Control"] = "no-cache, no-store, must-revalidate"
    r.headers["Pragma"] = "no-cache"
    r.headers["Expires"] = "0"
    r.headers['Cache-Control'] = 'public, max-age=0'
    return r

## Index Page

In [11]:
#default page of our web-app
@app.route('/')
@app.route('/home')
def home():
    return render_template('index.html')

## Facial Recognition Page

In [12]:
@app.route('/face_recog/')
def face_recog():
    return render_template('facerecog.html')

In [13]:
@app.route('/face_result')
def face_result():
    return recog()

In [14]:
@app.route('/detect_face/')
def detect_face():
    return render_template('face_result.html')

In [15]:
@app.route('/uploader', methods = ['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        file = request.files['file']
        text = request.form['text']
        # if user does not select file, browser also
        # submit a empty part without filename
        if file.filename == '':
            return redirect(url_for('face_recog'))
        if file and text:
            file_name = text + ".jpg"
            file.save(os.path.join(app.config['UPLOAD_FOLDER'], file_name))
            return redirect(url_for('face_recog'))
    else:
        return redirect(url_for('face_recog'))

## Object Detection

In [16]:
@app.route('/object_detection/')
def object_detection():
    result = main()
    return render_template('object_detection.html', text = result)

In [17]:
if __name__ == '__main__':
   app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [07/Apr/2021 17:48:29] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Apr/2021 17:48:30] "GET /static/css/styles.css HTTP/1.1" 304 -
127.0.0.1 - - [07/Apr/2021 17:48:30] "GET /js/scripts.js HTTP/1.1" 404 -
127.0.0.1 - - [07/Apr/2021 17:48:30] "GET /js/scripts.js HTTP/1.1" 404 -
127.0.0.1 - - [07/Apr/2021 17:48:30] "GET /static/assets/img/city.jpg HTTP/1.1" 304 -
127.0.0.1 - - [07/Apr/2021 17:48:50] "GET /object_detection/ HTTP/1.1" 200 -
127.0.0.1 - - [07/Apr/2021 17:48:50] "GET /static/css/styles.css HTTP/1.1" 304 -
127.0.0.1 - - [07/Apr/2021 17:48:50] "GET /object_detection/js/scripts.js HTTP/1.1" 404 -
127.0.0.1 - - [07/Apr/2021 17:48:50] "GET /static//loading.gif HTTP/1.1" 308 -
127.0.0.1 - - [07/Apr/2021 17:48:50] "GET /static/assets/img/city.jpg HTTP/1.1" 304 -
127.0.0.1 - - [07/Apr/2021 17:48:51] "GET /static/loading.gif HTTP/1.1" 304 -
127.0.0.1 - - [07/Apr/2021 17:48:56] "GET /static//output.jpg HTTP/1.1" 308 